In [17]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv
# !pip install scikit-learn

In [18]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [20]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# Correct Groq Model ID
model_name = "llama-3.1-8b-instant" 

# Test if it works
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, is this working?"}
    ],
    model=model_name,
)

print(chat_completion.choices[0].message.content)

Hello! It looks like it is working. I'm a large language model, and I'm here to help with any questions or tasks you have. What would you like to talk about or get assistance with today?


# Get LLM response

In [21]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome.


# Prompt engineering

## Structred output

In [22]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Italy?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [23]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

## input structuring

In [24]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
},
{
    "country": "France",
    "capital": "Paris"
},
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [25]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of thought)

> https://arxiv.org/pdf/2205.11916

In [26]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [27]:
259/2*8654+91072*33-12971

4113098.0

In [28]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1434921119
}


In [29]:
4113098.0 - 1431449.0

2681649.0

In [30]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [31]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


In [32]:
4113098.0 - 4113056

42.0

# RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [33]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

Since I'm not aware of any official information about the iPhone 16, I'll provide you with some general information about the latest iPhone models and some rumored features that might be included in future iPhone models.

However, I can tell you about the latest iPhone models, such as the iPhone 14 series, which includes:

1. iPhone 14
2. iPhone 14 Plus
3. iPhone 14 Pro
4. iPhone 14 Pro Max

Some of the notable features of the iPhone 14 series include:

1. Improved cameras with a new 48MP main camera on the Pro models
2. Faster A16 Bionic chip
3. Longer battery life
4. New colors and designs
5. Enhanced display with a higher refresh rate

As for the iPhone 16, there's no official information available yet. However, based on rumors and leaks, here are some potential features that might be included:

1. Improved cameras with a new periscope lens or a higher-resolution main camera
2. Faster A17 Bionic chip or a new chip design
3. Enhanced display with a higher refresh rate or a new OLED p

#### Giving Context to the unknown subject

In [34]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [35]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the provided information, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch display
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. Thinner bezels and a more durable Ceramic Shield

3. Upgraded A18 chip (A18 Pro for Pro models) for improved performance, including:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max

7. Switch to USB-C for faster charging and data transfer

8. Support for up to 2x faster video encoding in Pro models

9. Starting prices remain consis

#### Automatically extract context data from database

In [36]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [37]:
data = [iphone_16,samsung_s23]

In [38]:
user_prompt = """What's new in iphone 16?"""

In [40]:
embedding_client = OpenAI(
        api_key=os.getenv("GROQ_API_KEY"), 
        base_url=os.getenv("EMBEDDING_URL")
    )

In [41]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [46]:
#using local machine for embeddings

In [61]:
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import os

# --- EMBEDDING (LOCAL) ---
# Initialize the local embedding model once outside the function
local_embedding_model = SentenceTransformer('all-MiniLM-L6-v2') 

def get_embedding(model, text_input):
    # This is now the local model's encode method
    embedding_vector = model.encode(text_input).tolist()
    return embedding_vector

# --- LLM CLIENT (GROQ) ---
# Initialize the Groq client for chat completions
groq_client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)
groq_model = "llama-3.1-8b-instant"

# --- EXAMPLE USAGE ---
user_prompt = "What are the new features of the iPhone 16?"
# Get embeddings using the local model object
user_prompt_embeddings = get_embedding(local_embedding_model, user_prompt)
# ... then use groq_client for the chat completion ...

In [62]:
user_prompt_embeddings

[-0.027664165943861008,
 0.06676303595304489,
 0.0700257420539856,
 -0.05728081241250038,
 0.02660966105759144,
 0.011028705164790154,
 -0.03125479817390442,
 0.0009957123547792435,
 -0.05516740307211876,
 0.03669792041182518,
 0.09570624679327011,
 0.017850054427981377,
 -0.07627023756504059,
 0.07319768518209457,
 0.0464191660284996,
 -0.01411670632660389,
 0.1106327548623085,
 -0.0326492078602314,
 -0.04606301710009575,
 -0.06636357307434082,
 -0.050390951335430145,
 -0.08455319702625275,
 0.031811486929655075,
 0.019057875499129295,
 0.04801281541585922,
 0.03449780121445656,
 -0.06899016350507736,
 -0.08249595761299133,
 -0.013204301707446575,
 -0.04358138516545296,
 0.001232244074344635,
 0.08278965950012207,
 0.09351005405187607,
 0.002129138680174947,
 -0.09207350760698318,
 -0.11647623032331467,
 0.09164678305387497,
 0.005110481753945351,
 0.008118653669953346,
 -0.05999040976166725,
 -0.022250203415751457,
 -0.030106328427791595,
 -0.02097237855195999,
 0.06972958892583847,


In [63]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [64]:
# CORRECT (New way with 2 args)
# Note: We use 'local_embedding_model' instead of 'embedding_client' and 'model_name'
data_embeddings = [get_embedding(local_embedding_model, x) for x in data]

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. Make sure your 'data' variable is a LIST of strings (not just one string)
# If 'data' is currently just one big string, this loop won't work right.
# Assuming 'data' is like: data = ["Menu Item 1...", "Menu Item 2...", "Policies..."]

# 2. Regenerate the embeddings correctly (This creates a List of Lists)
data_embeddings = [get_embedding(local_embedding_model, x) for x in data]

# 3. Regenerate the user prompt embedding
user_prompt_embeddings = get_embedding(local_embedding_model, user_prompt)

# 4. Run the similarity check again
# Note: We wrap 'user_prompt_embeddings' in [] to make it 2D (1 row)
# 'data_embeddings' is already 2D (multiple rows)
data_similarity_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

print("Similarity calculation successful!")
print(f"Scores shape: {data_similarity_scores.shape}")

Similarity calculation successful!
Scores shape: (1, 2)


In [66]:
data_similarity_scores

array([[0.64775942, 0.40167337]])

In [67]:
closest_entry_index=data_similarity_scores.argmax()
closest_entry_index

0

In [68]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [69]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [70]:
user_prompt_with_data

'\n\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine

In [71]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The new features of the iPhone 16 include:

1. Larger displays:
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch display
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. Thinner bezels and a more durable Ceramic Shield

3. Powered by the new A18 chip (A18 Pro for the Pro models), offering:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera
   - Enhanced by Apple’s "Camera Control" button for more flexible photography options

5. Advanced audio features:
   - "Audio Mix," which uses machine learning to separate background sounds from speech

6. Extended battery life, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone

7. Switch to USB-C for faster charging and data transfer

8. Pro models now supp